In [1]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

from datetime import datetime, timedelta
import time 
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [3]:
%%time
# train_file = r'/kaggle/input/amex-agg-data-rev2/agg_train_all_rev2_rev.parquet'
train_file = r'amex\agg_train_all_rev3.parquet'
df=pd.read_parquet(train_file, engine='pyarrow')

Wall time: 39.9 s


In [4]:
id_feats = ['customer_ID']
date_col =  'S_2'
target_col = 'target'

# x_cols = list(set(all_cols)-set(['customer_ID', 'S_2', 'target']))
# len(x_cols)

# x_cols = ['P_2__last', 'P_2__min', 'P_2__mean', 'D_48__last', 'P_2__max', 'R_1__mean__log', 'D_48__mean', 'R_1__std', 'D_44__mean__log', 'B_2__last__log', 'B_2__last', 'R_1__max', 'P_2__first', 'D_44__last__log', 'R_1__range', 'B_18__last', 'B_9__last', 'B_3__last__log', 'D_61__last', 'B_2__mean', 'D_48__max', 'D_55__last', 'B_9__mean', 'B_3__mean__log', 'R_1__mean', 'B_33__last', 'D_44__last', 'D_44__max', 'B_18__mean', 'D_48__min', 'B_23__last__log', 'B_33__mean', 'B_3__last', 'B_33__last__log', 'B_7__last', 'B_11__last__log', 'D_75__last', 'D_58__last', 'B_2__last2max', 'B_20__last__log', 'B_23__last', 'R_10__mean__log', 'B_23__mean__log', 'B_2__mean__log', 'D_55__max', 'D_61__mean', 'B_3__max', 'B_18__last__log', 'B_22__mean__log', 'D_75__mean__log', 'B_2__min', 'B_9__last__log', 'D_75__last__log', 'D_44__mean', 'B_11__mean__log', 'R_1__last', 'B_20__mean__log', 'D_75__max', 'R_10__std', 'B_18__min', 'B_16__last', 'D_61__min', 'D_78__mean__log', 'D_48__first', 'B_9__mean__log', 'D_58__max', 'B_4__last', 'D_44__std', 'B_3__range', 'B_3__std', 'D_74__last', 'B_20__last', 'B_19__last__log', 'R_10__max', 'D_75__mean', 'R_10__range', 'D_44__range', 'B_19__mean__log', 'D_58__mean', 'B_22__max', 'B_19__last', 'D_74__mean__log', 'B_7__max', 'D_74__last__log', 'B_1__last', 'B_7__mean', 'B_23__max', 'B_37__last', 'R_1__last__log', 'B_4__last__log', 'R_2__mean__log', 'D_58__last__log', 'R_2__max', 'B_19__max', 'B_18__mean__log', 'B_3__mean', 'B_23__mean', 'R_2__std', 'B_33__mean__log', 'D_55__mean', 'D_74__max', 'B_40__last__log', 'D_58__mean__log', 'B_9__max', 'R_2__range', 'B_18__max', 'B_40__mean__log', 'B_11__last', 'B_4__mean__log', 'B_9__min', 'B_20__max', 'D_78__std', 'B_30=0.0', 'D_74__mean', 'B_22__mean', 'B_30__nunique', 'B_16__max', 'R_2__last', 'D_78__range', 'B_33__last2max', 'D_78__max', 'D_55__std', 'B_22__std', 'R_4__std', 'D_55__range', 'R_5__mean__log', 'B_18__last2max', 'D_55__last__log', 'R_4__mean__log', 'R_4__max', 'D_77__mean__log', 'B_22__range', 'B_1__mean', 'B_22__last', 'B_16__last__log', 'B_4__mean', 'R_4__range', 'B_37__mean', 'R_10__mean', 'R_8__mean__log', 'B_33__min', 'B_4__max', 'D_77__last__log', 'B_30=1.0', 'D_84__mean__log', 'D_75__min', 'B_4__last2max', 'B_19__mean', 'B_22__last__log', 'B_28__last2max', 'D_39__max', 'R_6__mean__log', 'D_41__range', 'B_11__mean', 'D_39__std', 'D_41__std', 'R_2__mean', 'D_58__min', 'B_14__last2max', 'D_39__range', 'B_19__range', 'S_7__max', 'D_55__mean__log', 'B_16__mean', 'B_7__min', 'B_11__last2max', 'B_2__max', 'S_7__mean', 'S_25__std', 'D_41__last', 'B_20__mean', 'D_41__max', 'B_1__max', 'B_37__max', 'S_3__max', 'B_16__last2max', 'S_25__range', 'D_44__first', 'P_3__min', 'S_3__mean', 'D_75__first', 'D_75__last2max', 'R_7__mean__log', 'B_9__first', 'P_3__mean', 'B_19__std', 'B_23__min', 'D_41__last__log', 'S_25__min', 'D_112__last__log', 'D_70__max', 'D_41__mean__log', 'R_4__mean', 'B_16__mean__log', 'D_74__min', 'B_18__first', 'B_33__max', 'B_11__max', 'D_70__mean__log', 'P_2__range', 'B_9__last2max', 'D_44__min', 'D_62__mean', 'D_78__last__log', 'P_2__std', 'D_70__std', 'B_6__last2max', 'B_4__min', 'S_7__last', 'S_25__mean', 'R_4__last', 'R_2__last__log', 'D_65__mean__log', 'B_23__range', 'B_2__first', 'B_38=2.0', 'D_112__last', 'D_62__max', 'R_3__mean', 'R_3__max', 'D_74__first', 'B_10__min', 'D_58__first', 'D_75__std', 'B_17__min', 'D_70__mean', 'B_25__last2max', 'D_78__last', 'D_70__range', 'B_20__last2max', 'D_75__range', 'B_23__last2max', 'P_3__last', 'S_3__last', 'D_112__last2max', 'B_8__min', 'D_55__min', 'D_39__last', 'D_62__last', 'D_78__mean', 'D_74__std', 'B_7__range', 'D_77__mean', 'S_15__max', 'R_8__std', 'B_37__last2max', 'B_23__std', 'D_74__range', 'B_38=4.0', 'B_1__min', 'B_8__mean', 'D_45__mean__log', 'S_7__range', 'D_58__last2max', 'B_7__first', 'R_8__max', 'R_8__range', 'D_77__max', 'B_8__first', 'R_15__std', 'D_70__last', 'B_37__min', 'R_15__max', 'D_77__last', 'R_15__range', 'S_7__std', 'B_7__std', 'D_74__last2max', 'B_23__first', 'R_15__mean__log', 'B_38__nunique', 'S_3__range', 'B_3__min', 'R_3__mean__log', 'D_43__mean', 'D_53__last__log', 'R_5__std', 'R_5__max', 'D_58__range', 'D_58__std', 'B_1__last2max', 'R_5__range', 'R_3__range', 'D_77__first', 'D_84__max', 'D_62__first', 'R_3__std', 'B_11__min', 'S_3__std', 'D_84__range', 'B_7__last2max', 'B_8__mean__log', 'B_3__last2max', 'B_28__mean__log', 'B_16__min', 'B_8__last', 'D_77__min', 'D_51__mean__log', 'D_84__std', 'D_70__last__log', 'B_33__first', 'D_45__last__log', 'S_15__range', 'B_38=5.0', 'B_20__range', 'D_43__max', 'D_53__mean__log', 'S_15__std', 'S_7__min', 'D_44__last2max', 'D_55__last2max', 'B_8__last__log', 'B_8__max', 'D_43__mean__log', 'D_62__min', 'D_39__mean', 'D_55__first', 'P_4__first', 'P_4__max', 'D_72__mean__log', 'B_3__first', 'D_84__last__log', 'R_15__mean', 'R_10__last', 'S_7__first', 'B_4__first', 'S_3__min', 'D_112__min', 'S_3__first', 'D_47__min', 'P_3__std', 'D_47__last', 'D_41__mean', 'D_39__mean__log', 'D_47__mean', 'P_4__mean__log', 'B_4__std', 'S_25__last', 'B_26__mean__log', 'D_81__mean__log', 'B_19__min', 'R_6__last__log', 'D_43__last', 'R_8__mean', 'B_10__last2max', 'D_47__first', 'B_9__std', 'B_20__std', 'B_1__first', 'D_45__first', 'D_45__max', 'R_5__last__log', 'B_40__min', 'D_43__last__log', 'B_37__first', 'D_47__max', 'R_16__mean__log', 'B_20__min', 'R_24__max', 'R_16__std', 'R_3__first', 'B_40__last2max', 'D_51__last__log', 'D_53__max', 'R_16__mean', 'D_53__last', 'D_54__last', 'R_24__range', 'R_24__std', 'D_54__last2max', 'D_65__last__log', 'B_4__range', 'D_43__std', 'B_19__first', 'D_45__mean', 'R_4__last__log', 'D_51__mean', 'D_39__last__log', 'R_3__last', 'B_9__range', 'R_20__mean__log', 'R_24__mean__log', 'R_10__last__log', 'B_28__last', 'R_27__last', 'R_27__min', 'D_51__last', 'D_84__last', 'R_5__mean', 'D_43__range', 'D_43__min', 'R_27__mean', 'R_14__mean__log', 'D_53__range', 'B_11__first', 'D_54__min', 'D_51__max', 'D_53__std', 'R_16__max', 'R_7__last__log', 'D_45__min', 'R_16__range', 'P_4__mean', 'B_22__first', 'D_45__last', 'D_79__mean__log', 'D_131__max', 'R_26__mean2std', 'B_16__std', 'D_54__range', 'D_131__mean__log', 'D_81__max', 'B_26__last__log', 'B_20__first', 'D_81__last', 'R_1__first', 'D_84__mean', 'B_19__last2max', 'D_72__max', 'D_131__last', 'S_15__mean', 'R_27__max', 'R_27__first', 'P_3__range', 'R_5__last', 'D_42__max', 'B_1__range', 'D_53__mean', 'D_112__mean', 'D_42__first', 'B_30=2.0', 'D_42__mean', 'B_37__range', 'B_11__range', 'B_17__mean2std', 'B_16__first', 'D_120__nunique', 'B_16__range', 'D_72__range', 'D_81__range', 'D_70__first', 'D_43__first', 'B_1__std', 'B_37__std', 'D_131__mean', 'D_52__min', 'D_89__mean__log', 'D_79__max', 'D_42__min', 'B_11__std', 'D_42__last', 'D_72__std', 'D_81__last__log', 'B_17__mean', 'D_89__std', 'D_81__std', 'D_54__std', 'D_89__max', 'R_13__max', 'D_89__range', 'R_8__last__log', 'R_13__std', 'D_79__last', 'D_120=0.0', 'D_112__range', 'D_72__last', 'R_13__range', 'D_52__mean', 'D_51__first', 'D_79__mean', 'D_72__last__log', 'R_3__last__log', 'S_8__min', 'D_106__mean2std', 'D_112__std', 'B_21__mean__log', 'R_8__last', 'B_28__min', 'D_130__last', 'D_42__mean2std', 'R_3__min', 'D_49__mean2std', 'D_52__last', 'R_24__mean', 'D_46__mean', 'S_25__last2max', 'D_130__max', 'D_62__last2max', 'D_120=1.0', 'D_127__max', 'D_127__mean__log', 'D_52__first', 'D_79__last__log', 'R_11__mean', 'D_131__last__log', 'D_51__min', 'B_25__last', 'D_130__mean__log', 'B_28__mean', 'D_53__mean2std', 'R_21__max', 'D_64=O', 'R_21__mean__log', 'S_11__min', 'S_16__mean__log', 'B_8__last2max', 'R_21__std', 'D_114=1.0', 'R_11__std', 'D_59__last', 'D_130__last__log', 'R_24__last', 'R_13__mean__log', 'D_130__mean', 'R_21__range', 'P_4__range', 'R_6__max', 'S_15__last', 'D_131__first', 'D_59__mean', 'D_133__mean__log', 'R_6__range', 'R_9__mean2std', 'D_81__mean', 'B_2__std', 'R_20__max', 'D_127__mean', 'D_52__max', 'D_70__min', 'R_21__mean', 'P_4__std', 'R_20__range', 'R_20__std', 'B_12__mean__log', 'B_25__min', 'D_135__mean2std', 'D_71__last__log', 'D_59__max', 'D_89__mean', 'D_137__mean2std', 'D_138__mean2std', 'D_72__mean', 'D_79__first', 'R_6__std', 'D_53__min', 'D_136__mean2std', 'D_132__mean2std', 'R_15__last', 'S_5__mean__log', 'D_112__mean__log', 'D_41__mean2std', 'D_116__nunique', 'D_134__mean2std', 'S_13__last2max', 'D_128__min', 'D_128__first', 'D_131__min', 'D_64__last=O', 'S_20__max', 'D_128__mean', 'D_51__std', 'R_11__mean__log', 'S_27__last__log', 'D_115__mean__log', 'S_27__last2max', 'P_3__first', 'B_2__range', 'S_8__last', 'D_128__mean__log', 'D_68=6.0', 'R_13__mean', 'R_11__max', 'D_79__std', 'B_12__last__log', 'S_20__std', 'D_127__last', 'D_128__max', 'D_79__min', 'B_25__mean', 'S_20__mean__log', 'D_128__last', 'D_59__min', 'R_11__range', 'D_53__first', 'D_127__range', 'S_20__range', 'D_118__max', 'D_115__mean', 'D_51__range', 'D_130__min', 'D_79__range', 'D_115__max', 'D_62__std', 'D_71__mean__log', 'D_62__range', 'D_115__last', 'D_119__max', 'D_68__nunique', 'B_20__mean2std', 'D_78__first', 'R_17__max', 'D_130__first', 'D_128__last__log', 'B_38=6.0', 'D_127__std', 'D_115__last__log', 'D_92__mean__log', 'B_17__last', 'R_17__std', 'B_24__mean__log', 'R_17__range', 'S_8__mean', 'R_12__last', 'D_118__mean', 'B_32__max', 'B_22__min', 'S_6__range', 'R_20__mean', 'R_12__min', 'D_119__mean', 'P_4__min', 'R_6__mean', 'B_32__range', 'D_118__mean__log', 'D_115__first', 'S_20__mean', 'D_48__std', 'D_118__last', 'S_8__last2max', 'P_4__last', 'B_28__max', 'D_115__min', 'D_119__last', 'D_119__mean__log', 'S_13__min', 'D_56__mean2std', 'D_54__mean', 'S_5__last__log', 'D_92__max', 'S_15__first', 'D_39__mean2std', 'B_32__std', 'S_6__last2max', 'S_22__max', 'D_121__max', 'S_6__std', 'D_121__last', 'D_118__first', 'R_6__last', 'R_12__range', 'D_133__last__log', 'D_121__mean', 'D_89__last', 'B_32__mean__log', 'D_127__first', 'D_119__first', 'D_122__mean', 'R_12__std', 'R_13__last', 'D_92__mean', 'D_127__last__log', 'D_59__first', 'D_91__mean__log', 'D_122__last', 'S_24__max', 'D_113__last', 'D_121__first', 'D_113__mean__log', 'D_42__range', 'B_5__mean__log', 'D_121__min', 'D_113__mean', 'B_38=1.0', 'B_24__std', 'D_48__range', 'D_113__min', 'R_21__last', 'R_17__mean', 'B_24__range', 'D_126=999.0', 'B_24__max', 'S_13__last', 'D_118__min', 'R_10__first', 'D_122__max', 'S_25__first', 'D_133__max', 'R_2__first', 'D_113__last__log', 'D_91__max', 'D_122__min', 'D_119__min', 'P_4__last__log', 'B_17__last2max', 'D_92__last', 'D_131__range', 'D_122__mean__log', 'D_118__last__log', 'S_23__mean2std', 'R_12__mean', 'R_17__mean__log', 'S_27__std', 'D_113__max', 'D_113__first', 'D_114=0.0', 'D_119__last__log', 'B_28__first', 'D_131__std', 'D_46__last', 'D_77__range', 'D_64__last=U', 'B_17__mean__log', 'R_14__last__log', 'R_25__std', 'R_20__last', 'D_133__last', 'B_31__nunique', 'S_11__std', 'R_25__max', 'R_22__max', 'D_114=999.0', 'D_120=999.0', 'D_117=999.0', 'D_116=999.0', 'D_122__first', 'R_25__range', 'R_22__range', 'D_114__nunique', 'B_17__std', 'D_133__mean', 'B_17__range', 'B_17__last__log', 'S_22__mean', 'S_27__range', 'D_64__nunique', 'B_3__mean2std', 'R_20__last__log', 'R_1__min', 'S_11__range', 'S_24__mean', 'R_22__std', 'S_13__std', 'D_65__mean2std', 'B_38=7.0', 'S_6__min', 'D_129__first', 'D_42__std', 'D_133__range', 'B_26__mean2std', 'D_129__mean', 'D_122__last__log', 'B_24__mean', 'D_41__first', 'D_92__range', 'D_129__mean__log', 'D_46__first', 'D_92__first', 'B_24__last__log', 'D_112__max', 'D_92__std', 'D_68=999.0', 'D_91__std', 'D_91__range', 'B_24__last', 'D_129__max', 'D_91__mean', 'S_27__mean2std', 'D_53__last2max', 'R_25__mean__log', 'D_64=NA', 'D_133__std', 'R_24__last__log', 'B_31=0', 'S_8__first', 'D_129__min', 'D_77__std', 'D_106__last2max', 'S_20__last', 'S_26__mean__log', 'R_21__first', 'R_1__mean2std', 'D_72__first', 'D_92__last__log', 'D_52__last2max', 'D_129__last', 'B_5__last__log', 'S_13__last__log', 'R_25__mean', 'D_116=0.0', 'R_22__mean__log', 'D_41__min', 'R_7__std', 'D_140__max', 'B_32__last', 'R_7__max', 'R_15__last__log', 'D_140__mean__log', 'D_130__last2max', 'R_7__range', 'D_91__last', 'D_129__last__log', 'B_21__max', 'B_21__std', 'D_41__last2max', 'D_140__mean', 'B_19__mean2std', 'B_32__mean', 'B_21__range', 'R_6__mean2std', 'R_19__last', 'D_128__last2max', 'R_22__mean', 'D_56__mean', 'D_143__last', 'D_139__last', 'D_64=U', 'D_103__mean', 'D_145__mean__log', 'R_5__mean2std', 'S_6__last', 'B_33__std', 'B_41__max', 'D_140__last', 'D_104__mean', 'B_41__mean__log', 'B_41__range', 'S_11__last2max', 'R_9__std', 'B_22__last2max', 'D_103__last', 'D_56__last', 'D_56__min', 'D_56__first', 'D_141__last', 'R_4__first', 'R_7__mean', 'D_104__last', 'B_14__min', 'D_56__max', 'D_143__min', 'D_139__min', 'B_41__std', 'D_143__mean', 'D_139__mean', 'B_4__mean2std']
x_cols = ['P_2__last', 'P_2__min', 'P_2__mean', 'D_48__last', 'P_2__max', 'R_1__mean__log', 'D_48__mean', 'R_1__std', 'D_44__mean__log', 'B_2__last__log', 'B_2__last', 'R_1__max', 'P_2__first', 'D_44__last__log', 'R_1__range', 'B_18__last', 'B_9__last', 'B_3__last__log', 'D_61__last', 'B_2__mean', 'D_48__max', 'D_55__last', 'B_9__mean', 'B_3__mean__log', 'R_1__mean', 'B_33__last', 'D_44__last', 'D_44__max', 'B_18__mean', 'D_48__min', 'B_23__last__log', 'B_33__mean', 'B_3__last', 'B_33__last__log', 'B_7__last', 'B_11__last__log', 'D_75__last', 'D_58__last', 'B_2__last2max', 'B_20__last__log', 'B_23__last', 'R_10__mean__log', 'B_23__mean__log', 'B_2__mean__log', 'D_55__max', 'D_61__mean', 'B_3__max', 'B_18__last__log', 'B_22__mean__log', 'D_75__mean__log', 'B_2__min', 'B_9__last__log', 'D_75__last__log', 'D_44__mean', 'B_11__mean__log', 'R_1__last', 'B_20__mean__log', 'D_75__max', 'R_10__std', 'B_18__min', 'B_16__last', 'D_61__min', 'D_78__mean__log', 'D_48__first', 'B_9__mean__log', 'D_58__max', 'B_4__last', 'D_44__std', 'B_3__range', 'B_3__std', 'D_74__last', 'B_20__last', 'B_19__last__log', 'R_10__max', 'D_75__mean', 'R_10__range', 'D_44__range', 'B_19__mean__log', 'D_58__mean', 'B_22__max', 'B_19__last', 'D_74__mean__log', 'B_7__max', 'D_74__last__log', 'B_1__last', 'B_7__mean', 'B_23__max', 'B_37__last', 'R_1__last__log', 'B_4__last__log', 'R_2__mean__log', 'D_58__last__log', 'R_2__max', 'B_19__max', 'B_18__mean__log', 'B_3__mean', 'B_23__mean', 'R_2__std', 'B_33__mean__log', 'D_55__mean', 'D_74__max', 'B_40__last__log', 'D_58__mean__log', 'B_9__max', 'R_2__range', 'B_18__max', 'B_40__mean__log', 'B_11__last', 'B_4__mean__log', 'B_9__min', 'B_20__max', 'D_78__std', 'B_30=0.0', 'D_74__mean', 'B_22__mean', 'B_30__nunique', 'B_16__max', 'R_2__last', 'D_78__range', 'B_33__last2max', 'D_78__max', 'D_55__std', 'B_22__std', 'R_4__std', 'D_55__range', 'R_5__mean__log', 'B_18__last2max', 'D_55__last__log', 'R_4__mean__log', 'R_4__max', 'D_77__mean__log', 'B_22__range', 'B_1__mean', 'B_22__last', 'B_16__last__log', 'B_4__mean', 'R_4__range', 'B_37__mean', 'R_10__mean', 'R_8__mean__log', 'B_33__min', 'B_4__max', 'D_77__last__log', 'B_30=1.0', 'D_84__mean__log', 'D_75__min', 'B_4__last2max', 'B_19__mean', 'B_22__last__log', 'B_28__last2max', 'D_39__max', 'R_6__mean__log', 'D_41__range', 'B_11__mean', 'D_39__std', 'D_41__std', 'R_2__mean', 'D_58__min', 'B_14__last2max', 'D_39__range', 'B_19__range', 'S_7__max', 'D_55__mean__log', 'B_16__mean', 'B_7__min', 'B_11__last2max', 'B_2__max', 'S_7__mean', 'S_25__std', 'D_41__last', 'B_20__mean', 'D_41__max', 'B_1__max', 'B_37__max', 'S_3__max', 'B_16__last2max', 'S_25__range', 'D_44__first', 'P_3__min', 'S_3__mean', 'D_75__first', 'D_75__last2max', 'R_7__mean__log', 'B_9__first', 'P_3__mean', 'B_19__std', 'B_23__min', 'D_41__last__log', 'S_25__min', 'D_112__last__log', 'D_70__max', 'D_41__mean__log', 'R_4__mean', 'B_16__mean__log', 'D_74__min', 'B_18__first', 'B_33__max', 'B_11__max', 'D_70__mean__log', 'P_2__range', 'B_9__last2max', 'D_44__min', 'D_62__mean', 'D_78__last__log', 'P_2__std', 'D_70__std', 'B_6__last2max', 'B_4__min', 'S_7__last', 'S_25__mean', 'R_4__last', 'R_2__last__log', 'D_65__mean__log', 'B_23__range', 'B_2__first', 'B_38=2.0', 'D_112__last', 'D_62__max', 'R_3__mean', 'R_3__max', 'D_74__first', 'B_10__min', 'D_58__first', 'D_75__std', 'B_17__min', 'D_70__mean', 'B_25__last2max', 'D_78__last', 'D_70__range', 'B_20__last2max', 'D_75__range', 'B_23__last2max', 'P_3__last', 'S_3__last', 'D_112__last2max', 'B_8__min', 'D_55__min', 'D_39__last', 'D_62__last', 'D_78__mean', 'D_74__std', 'B_7__range', 'D_77__mean', 'S_15__max', 'R_8__std', 'B_37__last2max', 'B_23__std', 'D_74__range', 'B_38=4.0', 'B_1__min', 'B_8__mean', 'D_45__mean__log', 'S_7__range', 'D_58__last2max', 'B_7__first', 'R_8__max', 'R_8__range', 'D_77__max', 'B_8__first', 'R_15__std', 'D_70__last', 'B_37__min', 'R_15__max', 'D_77__last', 'R_15__range', 'S_7__std', 'B_7__std', 'D_74__last2max', 'B_23__first', 'R_15__mean__log', 'B_38__nunique', 'S_3__range', 'B_3__min', 'R_3__mean__log', 'D_43__mean', 'D_53__last__log', 'R_5__std', 'R_5__max', 'D_58__range', 'D_58__std', 'B_1__last2max', 'R_5__range', 'R_3__range', 'D_77__first', 'D_84__max', 'D_62__first', 'R_3__std', 'B_11__min', 'S_3__std', 'D_84__range', 'B_7__last2max', 'B_8__mean__log', 'B_3__last2max', 'B_28__mean__log', 'B_16__min', 'B_8__last', 'D_77__min', 'D_51__mean__log', 'D_84__std', 'D_70__last__log', 'B_33__first', 'D_45__last__log', 'S_15__range', 'B_38=5.0', 'B_20__range', 'D_43__max', 'D_53__mean__log', 'S_15__std', 'S_7__min', 'D_44__last2max', 'D_55__last2max', 'B_8__last__log', 'B_8__max', 'D_43__mean__log', 'D_62__min', 'D_39__mean', 'D_55__first', 'P_4__first', 'P_4__max', 'D_72__mean__log', 'B_3__first', 'D_84__last__log', 'R_15__mean', 'R_10__last', 'S_7__first', 'B_4__first', 'S_3__min', 'D_112__min', 'S_3__first', 'D_47__min', 'P_3__std', 'D_47__last', 'D_41__mean', 'D_39__mean__log', 'D_47__mean', 'P_4__mean__log', 'B_4__std', 'S_25__last', 'B_26__mean__log', 'D_81__mean__log', 'B_19__min', 'R_6__last__log', 'D_43__last', 'R_8__mean', 'B_10__last2max', 'D_47__first', 'B_9__std', 'B_20__std', 'B_1__first', 'D_45__first', 'D_45__max', 'R_5__last__log', 'B_40__min', 'D_43__last__log', 'B_37__first', 'D_47__max', 'R_16__mean__log', 'B_20__min', 'R_24__max', 'R_16__std', 'R_3__first', 'B_40__last2max', 'D_51__last__log', 'D_53__max', 'R_16__mean', 'D_53__last', 'D_54__last', 'R_24__range', 'R_24__std', 'D_54__last2max', 'D_65__last__log', 'B_4__range', 'D_43__std', 'B_19__first', 'D_45__mean', 'R_4__last__log', 'D_51__mean', 'D_39__last__log', 'R_3__last', 'B_9__range', 'R_20__mean__log', 'R_24__mean__log', 'R_10__last__log', 'B_28__last', 'R_27__last', 'R_27__min', 'D_51__last', 'D_84__last', 'R_5__mean', 'D_43__range', 'D_43__min', 'R_27__mean', 'R_14__mean__log', 'D_53__range', 'B_11__first', 'D_54__min', 'D_51__max', 'D_53__std', 'R_16__max', 'R_7__last__log', 'D_45__min', 'R_16__range', 'P_4__mean', 'B_22__first', 'D_45__last', 'D_79__mean__log', 'D_131__max', 'R_26__mean2std', 'B_16__std', 'D_54__range', 'D_131__mean__log', 'D_81__max', 'B_26__last__log', 'B_20__first', 'D_81__last', 'R_1__first', 'D_84__mean', 'B_19__last2max', 'D_72__max', 'D_131__last', 'S_15__mean', 'R_27__max', 'R_27__first', 'P_3__range', 'R_5__last', 'D_42__max', 'B_1__range', 'D_53__mean', 'D_112__mean', 'D_42__first', 'B_30=2.0', 'D_42__mean', 'B_37__range', 'B_11__range', 'B_17__mean2std', 'B_16__first', 'D_120__nunique', 'B_16__range', 'D_72__range', 'D_81__range', 'D_70__first', 'D_43__first', 'B_1__std', 'B_37__std', 'D_131__mean', 'D_52__min', 'D_89__mean__log', 'D_79__max', 'D_42__min', 'B_11__std', 'D_42__last', 'D_72__std', 'D_81__last__log', 'B_17__mean', 'D_89__std', 'D_81__std', 'D_54__std', 'D_89__max', 'R_13__max', 'D_89__range', 'R_8__last__log', 'R_13__std', 'D_79__last', 'D_120=0.0', 'D_112__range', 'D_72__last', 'R_13__range', 'D_52__mean', 'D_51__first', 'D_79__mean', 'D_72__last__log', 'R_3__last__log', 'S_8__min', 'D_106__mean2std', 'D_112__std', 'B_21__mean__log', 'R_8__last', 'B_28__min', 'D_130__last', 'D_42__mean2std', 'R_3__min', 'D_49__mean2std', 'D_52__last', 'R_24__mean', 'D_46__mean', 'S_25__last2max', 'D_130__max', 'D_62__last2max', 'D_120=1.0', 'D_127__max', 'D_127__mean__log', 'D_52__first', 'D_79__last__log', 'R_11__mean', 'D_131__last__log', 'D_51__min', 'B_25__last', 'D_130__mean__log', 'B_28__mean', 'D_53__mean2std', 'R_21__max', 'D_64=O', 'R_21__mean__log', 'S_11__min', 'S_16__mean__log', 'B_8__last2max', 'R_21__std', 'D_114=1.0', 'R_11__std', 'D_59__last', 'D_130__last__log', 'R_24__last', 'R_13__mean__log', 'D_130__mean', 'R_21__range', 'P_4__range', 'R_6__max', 'S_15__last', 'D_131__first', 'D_59__mean', 'D_133__mean__log', 'R_6__range', 'R_9__mean2std', 'D_81__mean', 'B_2__std', 'R_20__max', 'D_127__mean', 'D_52__max', 'D_70__min', 'R_21__mean', 'P_4__std', 'R_20__range', 'R_20__std', 'B_12__mean__log', 'B_25__min', 'D_135__mean2std', 'D_71__last__log', 'D_59__max', 'D_89__mean', 'D_137__mean2std', 'D_138__mean2std', 'D_72__mean', 'D_79__first', 'R_6__std', 'D_53__min', 'D_136__mean2std', 'D_132__mean2std', 'R_15__last', 'S_5__mean__log', 'D_112__mean__log', 'D_41__mean2std', 'D_116__nunique', 'D_134__mean2std', 'S_13__last2max', 'D_128__min', 'D_128__first', 'D_131__min', 'D_64__last=O', 'S_20__max', 'D_128__mean', 'D_51__std', 'R_11__mean__log', 'S_27__last__log', 'D_115__mean__log', 'S_27__last2max', 'P_3__first', 'B_2__range', 'S_8__last', 'D_128__mean__log', 'D_68=6.0', 'R_13__mean', 'R_11__max', 'D_79__std', 'B_12__last__log', 'S_20__std', 'D_127__last', 'D_128__max', 'D_79__min', 'B_25__mean', 'S_20__mean__log', 'D_128__last', 'D_59__min', 'R_11__range', 'D_53__first', 'D_127__range', 'S_20__range', 'D_118__max', 'D_115__mean', 'D_51__range', 'D_130__min', 'D_79__range', 'D_115__max', 'D_62__std', 'D_71__mean__log', 'D_62__range', 'D_115__last', 'D_119__max', 'D_68__nunique', 'B_20__mean2std', 'D_78__first', 'R_17__max', 'D_130__first', 'D_128__last__log', 'B_38=6.0', 'D_127__std', 'D_115__last__log', 'D_92__mean__log', 'B_17__last', 'R_17__std', 'B_24__mean__log', 'R_17__range', 'S_8__mean', 'R_12__last', 'D_118__mean', 'B_32__max', 'B_22__min', 'S_6__range', 'R_20__mean', 'R_12__min', 'D_119__mean', 'P_4__min', 'R_6__mean', 'B_32__range', 'D_118__mean__log', 'D_115__first', 'S_20__mean', 'D_48__std', 'D_118__last', 'S_8__last2max', 'P_4__last', 'B_28__max', 'D_115__min', 'D_119__last', 'D_119__mean__log', 'S_13__min', 'D_56__mean2std', 'D_54__mean', 'S_5__last__log', 'D_92__max', 'S_15__first', 'D_39__mean2std', 'B_32__std', 'S_6__last2max', 'S_22__max', 'D_121__max', 'S_6__std', 'D_121__last', 'D_118__first', 'R_6__last', 'R_12__range', 'D_133__last__log', 'D_121__mean', 'D_89__last', 'B_32__mean__log', 'D_127__first', 'D_119__first', 'D_122__mean', 'R_12__std', 'R_13__last', 'D_92__mean', 'D_127__last__log', 'D_59__first', 'D_91__mean__log', 'D_122__last', 'S_24__max', 'D_113__last', 'D_121__first', 'D_113__mean__log', 'D_42__range', 'B_5__mean__log', 'D_121__min', 'D_113__mean', 'B_38=1.0', 'B_24__std', 'D_48__range', 'D_113__min', 'R_21__last', 'R_17__mean', 'B_24__range', 'D_126=999.0', 'B_24__max', 'S_13__last', 'D_118__min', 'R_10__first', 'D_122__max', 'S_25__first', 'D_133__max', 'R_2__first', 'D_113__last__log', 'D_91__max', 'D_122__min', 'D_119__min', 'P_4__last__log', 'B_17__last2max', 'D_92__last', 'D_131__range', 'D_122__mean__log', 'D_118__last__log', 'S_23__mean2std', 'R_12__mean', 'R_17__mean__log', 'S_27__std', 'D_113__max', 'D_113__first', 'D_114=0.0', 'D_119__last__log', 'B_28__first', 'D_131__std', 'D_46__last', 'D_77__range', 'D_64__last=U', 'B_17__mean__log', 'R_14__last__log', 'R_25__std', 'R_20__last', 'D_133__last', 'B_31__nunique', 'S_11__std', 'R_25__max', 'R_22__max', 'D_114=999.0', 'D_120=999.0', 'D_117=999.0', 'D_116=999.0', 'D_122__first', 'R_25__range', 'R_22__range', 'D_114__nunique', 'B_17__std', 'D_133__mean', 'B_17__range', 'B_17__last__log', 'S_22__mean', 'S_27__range', 'D_64__nunique', 'B_3__mean2std', 'R_20__last__log', 'R_1__min', 'S_11__range', 'S_24__mean', 'R_22__std', 'S_13__std', 'D_65__mean2std', 'B_38=7.0', 'S_6__min', 'D_129__first', 'D_42__std', 'D_133__range', 'B_26__mean2std', 'D_129__mean', 'D_122__last__log', 'B_24__mean', 'D_41__first', 'D_92__range', 'D_129__mean__log', 'D_46__first', 'D_92__first', 'B_24__last__log', 'D_112__max', 'D_92__std', 'D_68=999.0', 'D_91__std', 'D_91__range', 'B_24__last', 'D_129__max', 'D_91__mean', 'S_27__mean2std', 'D_53__last2max', 'R_25__mean__log', 'D_64=NA', 'D_133__std', 'R_24__last__log', 'B_31=0', 'S_8__first', 'D_129__min', 'D_77__std', 'D_106__last2max', 'S_20__last', 'S_26__mean__log', 'R_21__first', 'R_1__mean2std', 'D_72__first', 'D_92__last__log', 'D_52__last2max', 'D_129__last', 'B_5__last__log', 'S_13__last__log', 'R_25__mean', 'D_116=0.0', 'R_22__mean__log', 'D_41__min', 'R_7__std', 'D_140__max', 'B_32__last', 'R_7__max', 'R_15__last__log', 'D_140__mean__log', 'D_130__last2max', 'R_7__range', 'D_91__last', 'D_129__last__log', 'B_21__max', 'B_21__std', 'D_41__last2max', 'D_140__mean', 'B_19__mean2std', 'B_32__mean', 'B_21__range', 'R_6__mean2std', 'R_19__last', 'D_128__last2max', 'R_22__mean', 'D_56__mean', 'D_143__last', 'D_139__last', 'D_64=U', 'D_103__mean', 'D_145__mean__log', 'R_5__mean2std', 'S_6__last', 'B_33__std', 'B_41__max', 'D_140__last', 'D_104__mean', 'B_41__mean__log', 'B_41__range', 'S_11__last2max', 'R_9__std', 'B_22__last2max', 'D_103__last', 'D_56__last', 'D_56__min', 'D_56__first', 'D_141__last', 'R_4__first', 'R_7__mean', 'D_104__last', 'B_14__min', 'D_56__max', 'D_143__min', 'D_139__min', 'B_41__std', 'D_143__mean', 'D_139__mean', 'B_4__mean2std', 'R_9__range', 'D_107__mean__log', 'R_17__last', 'D_103__min', 'D_107__mean', 'D_107__last', 'D_104__min', 'B_41__last', 'D_107__last__log', 'D_141__min', 'D_141__mean', 'D_139__mean__log', 'D_103__last__log', 'D_143__mean__log', 'D_143__max', 'R_16__last', 'D_84__first', 'D_139__max', 'D_136__min', 'D_104__last__log', 'R_2__mean2std', 'D_141__mean__log', 'D_91__first', 'D_107__min', 'B_25__first', 'S_13__mean', 'D_103__mean__log', 'D_104__mean__log', 'P_3__max', 'B_21__mean', 'R_19__max', 'S_6__last__log', 'D_136__std', 'D_65__range', 'D_138__max', 'D_139__last__log', 'D_138__mean', 'D_61__max', 'D_141__max', 'D_65__max', 'D_136__last2max', 'D_143__first', 'D_139__first', 'D_141__last__log', 'D_143__last__log', 'B_31=1', 'B_23__mean2std', 'D_107__max', 'S_23__mean', 'D_145__last', 'D_138__last2max', 'D_145__last__log', 'R_7__mean2std', 'B_25__max', 'S_20__first', 'B_28__mean2std', 'D_141__first', 'D_103__max', 'D_60__last__log', 'D_136__range', 'D_84__mean2std', 'R_16__first', 'D_91__last__log', 'D_76__mean2std', 'D_104__max', 'D_103__first', 'P_3__last2max', 'R_19__mean__log', 'D_50__mean2std', 'B_21__last__log', 'D_60__last2max', 'D_104__first', 'D_51__last2max', 'S_13__range', 'D_130__range', 'D_145__mean', 'D_130__std', 'D_138__last', 'D_145__max', 'D_145__min', 'B_11__mean2std', 'D_65__last', 'D_69__mean__log', 'S_26__last__log', 'D_60__max', 'D_112__first', 'D_117=4.0', 'D_107__first', 'D_133__first', 'S_23__last', 'D_138__mean__log', 'D_83__mean__log', 'S_25__mean2std', 'D_60__range', 'D_127__min', 'days', 'B_6__min', 'D_81__first', 'R_19__mean', 'B_14__last', 'S_16__last__log', 'S_11__mean', 'D_145__first', 'D_46__min', 'B_7__mean2std', 'D_138__first', 'B_41__mean', 'D_39__last2max', 'D_123__mean__log', 'D_126=1.0', 'D_72__min', 'S_2=count', 'B_33__range', 'D_140__first', 'D_61__first', 'D_44__mean2std', 'B_31__last', 'D_92__min', 'R_7__last', 'R_10__last2max', 'R_16__last__log', 'D_59__std', 'D_60__std', 'R_4__mean2std', 'B_38=3.0', 'D_140__std', 'S_16__std', 'D_140__range', 'B_37__mean2std', 'B_1__mean2std', 'D_138__min', 'D_117__nunique']


len(x_cols)

977

In [5]:
%%time
df = df[x_cols + [target_col]].copy(deep=True)

Wall time: 9.41 s


## spFSR feature selection

In [6]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

from datetime import datetime, timedelta
import time 
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [7]:
import pyarrow.parquet as pq
import pyarrow as pa

In [8]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [9]:
df[x_cols].isna().sum()[df[x_cols].isna().sum()>0]

Series([], dtype: int64)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 978 entries, P_2__last to target
dtypes: float32(963), int32(13), uint8(2)
memory usage: 1.7 GB


In [11]:
    
def save_log(log_file, log_message, verbose=False):
    f = open(log_file, "a")
    f.write(log_message)
    f.close()
    
    if verbose:
        print(log_message)

In [12]:
log_file='amex/spfsr-rev3data-top977corrfeats.txt'

In [13]:

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from spFSR import SpFSR


In [ ]:
n_samples = 35000

for seed in range(1, 100):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
#     wrapper = DecisionTreeClassifier(max_depth=20, random_state=seed)
    wrapper = LogisticRegression(random_state=seed)
#     wrapper = SVC(random_state=seed)
    
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=wrapper)

    sp_run = sp_engine.run(num_features=0, n_samples_max=n_samples, print_freq=10, iter_max=50)  

    sp_results = sp_run.results

#     print('Best value:', sp_results.get('selected_ft_score_mean'))
#     print('Indices of selected features: ', sp_results.get('selected_features'))
#     print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
#     print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    save_log(log_file, f'{sel_feats},\n\n')
    print(sel_feats)

In [ ]:
n_samples = 45000

for seed in range(15, 20):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier(random_state=seed))

    sp_run = sp_engine.run(num_features=150, n_samples_max=n_samples, print_freq=5, iter_max=10)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)

In [ ]:
n_samples = 45000

for seed in range(20):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier(random_state=1))

    sp_run = sp_engine.run(num_features=100, n_samples_max=n_samples, print_freq=5, iter_max=20)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)

In [ ]:
n_samples = 45000

for seed in range(20):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier(random_state=1))

    sp_run = sp_engine.run(num_features=100, n_samples_max=n_samples, print_freq=5, iter_max=20)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)

In [ ]:
n_samples = 45000

for seed in range(20):
    df_ = df.sample(n=n_samples, replace=False, random_state=seed)
    x=df_[x_cols]
    y=df_['target']
    
    
    # set the engine parameters
    #sp_engine = SpFSR(x.values, y.values, pred_type='r', scoring='r2', wrapper=DecisionTreeRegressor(random_state=1))
    sp_engine = SpFSR(x.values, y.values, pred_type='c', scoring='roc_auc', wrapper=DecisionTreeClassifier)

    sp_run = sp_engine.run(num_features=100, n_samples_max=n_samples, print_freq=5, iter_max=20)  

    sp_results = sp_run.results

    print('Best value:', sp_results.get('selected_ft_score_mean'))
    print('Indices of selected features: ', sp_results.get('selected_features'))
    print('Importance of selected features: ', sp_results.get('selected_ft_importance').round(3))
    print('Total iterations for the optimal feature set:', sp_results.get('total_iter_for_opt'))

    sel_feats=list(np.array(x_cols)[sp_results.get('selected_features')])
    print(sel_feats)